In [1]:
import pandas as pd
import os
# Your BAM file path
bam_file = "Puck_190926_03.bam"

In [13]:
os.chdir("/mnt/disk2/yue_data/data/fastq/slideseq/Puck_200115_08/")

In [4]:
def get_tags_from_bam(bamfile_path, gene_tag, UMI_tag,sub_props):
    # List to store tags
    gene_count_dict = {sub_prop: {} for sub_prop in sub_props}
    res_cnt = {sub_prop: Counter() for sub_prop in sub_props}
    # Open the BAM file
    idx = 0
    with pysam.AlignmentFile(bamfile_path, "rb",require_index=False) as bamfile:
        # Iterate over each read
        for read in bamfile:
            idx += 1
            gene = read.get_tag(gene_tag) if read.has_tag(gene_tag) else None
            bc=read.get_tag("BC") if read.has_tag(gene_tag) else None
            UMI = read.get_tag('XM') if read.has_tag('XM') else None
            for sub_prop in sub_props:
                random_num = random.randint(0, 99)
                if random_num < sub_prop * 100:
                    # Check if the tag is present in the read
                    if (gene is not None) and (bc is not None) and (UMI is not None):
                        # Add the tag value to the list
                        if gene not in gene_count_dict[sub_prop]:
                            gene_count_dict[sub_prop][gene] = {}
                        gene_count_dict[sub_prop][gene].setdefault(bc, set()).add(UMI)
                        res_cnt[sub_prop]["transcriptomic"] += 1 
                    
            if idx % 1000000 ==0:
                print(f"Total reads processed: {idx}")
                print(res_cnt[sub_prop])
            #if idx == 1000000:
             #   break
        for sub_prop in sub_props:
            for gene in gene_count_dict[sub_prop]:
                gene_count_dict[sub_prop][gene] = {bc: len(gene_count_dict[sub_prop][gene][bc]) for bc in gene_count_dict[sub_prop][gene]}

    return gene_count_dict,res_cnt


In [5]:
def write_mtx(gene_count_dict,out_dir):
    """
    gene_count_dict: dict in dict {gene_id: {barcode:UMI_count}, ... }
    """
    gene_list = []
    bc_list = []
    for ge in gene_count_dict:
        gene_list.append(ge)
        for bc in gene_count_dict[ge]:
            bc_list.append(bc)
    total_cnt = len(bc_list)
    bc_list = list(set(bc_list))
    print(bc_list[:10])
    print(gene_list[:10])
    probe_dict = dict([(pid, ix+1) for ix,pid in enumerate(gene_list) ])  # 1-based indexing
    bc_dict = dict([(bc, ix+1) for ix, bc in enumerate(bc_list)])  # 1-based indexing
    out_feature = gzip.open(os.path.join(out_dir,"digital_expression_features.tsv.gz"),"wb")
    for ge in gene_list:
        out_feature.write(ge.encode('utf-8') + b'\n')
    out_feature.close()
    out_barcode = gzip.open(os.path.join(out_dir,"digital_expression_barcodes.tsv.gz"),"wb")
    for bc in bc_list:
        out_barcode.write(bytes(bc+'\n',"UTF-8"))
    out_barcode.close()
    out_mtx = gzip.open(os.path.join(out_dir,"digital_expression_matrix.mtx.gz"),"wb")
    out_mtx.write(b'%%MatrixMarket matrix coordinate integer general\n%\n')
    out_mtx.write(bytes("{}\t{}\t{}\n".format(len(gene_list),len(bc_list),total_cnt),"UTF-8" ))
    for ge in gene_list:
        for bc in gene_count_dict[ge]:
            out_mtx.write(bytes("{}\t{}\t{}\n".format(probe_dict[ge],bc_dict[bc],gene_count_dict[ge][bc]),"UTF-8") )
    out_mtx.close()

In [14]:
#8551639 for eb, 9974049 for hp1, 5087558 for hp2
sub_props = [round(5000000/5087558,2)]
sub_props

[0.98]

In [15]:
from collections import Counter
import gzip
import os
import random
import pysam
if __name__ == "__main__":
    gene_tag = "XG"
    UMI_tag = "XM"
    bamfile_path = "Puck_200115_08_hippo_clean.bam"  # replace with your bam file path
    gene_count_dict,res_cnt = get_tags_from_bam(bamfile_path, gene_tag, UMI_tag, sub_props)

[E::idx_find_and_load] Could not retrieve index file for 'Puck_200115_08_hippo_clean.bam'


Total reads processed: 1000000
Counter({'transcriptomic': 878928})
Total reads processed: 2000000
Counter({'transcriptomic': 1736561})
Total reads processed: 3000000
Counter({'transcriptomic': 2521900})
Total reads processed: 4000000
Counter({'transcriptomic': 3390483})
Total reads processed: 5000000
Counter({'transcriptomic': 4239926})
Total reads processed: 6000000
Counter({'transcriptomic': 4986121})
Total reads processed: 7000000
Counter({'transcriptomic': 4986121})


In [17]:
os.getcwd()

'/mnt/disk2/yue_data/data/fastq/slideseq/Puck_200115_08'

In [16]:
for prop in sub_props:
    out_dir = f"./output/{prop}"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    write_mtx(gene_count_dict[prop],out_dir)

['ATGTAACCCCTTAG', 'ATATGCCGCGATAG', 'CCTGAGGAAGCGAT', 'TATGTTTCTAGCTT', 'GTTCAGCCTAATAG', 'TCCTGGAGATACGC', 'AACCCCGTCATCAT', 'AGGGGCCGTTCTAG', 'AACTTACGAATCCG', 'GGATAGCAAGTTTA']
['9230019H11Rik', 'H60c', 'Gm26752', 'Gm26752,Ppp1r14c', 'Ppp1r14c', 'Iyd', 'Plekhg1', 'Plekhg1,Mthfd1l', 'Mthfd1l', 'Akap12']
